In [1]:
import warnings
warnings.simplefilter("ignore")
import pickle
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm_notebook
from collections import namedtuple

### Загрузка таблицы с адресами

In [43]:
atm = pickle.load(open('mta.dat','rb'))

In [44]:
atm = atm[atm.atm_caid.isin({'VB24',               
                            'Alfa Iss',           
                            'GAZPROMBANK',        
                            'Alfa Acq' })]

In [45]:
atm.columns  = ['atm_caid','atm_terminal','full_address']

In [46]:
atm['address'] = atm.full_address.apply(lambda x: x[:23].strip())

In [47]:
atm['city'] = atm.full_address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

In [48]:
def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

In [49]:
atm.city = atm.city.map(transform)

In [50]:
atm['address_rus'] = ''
atm['lat'] = np.nan
atm['long'] = np.nan

In [51]:
Station = namedtuple('Station', 'name distance')
Organization = namedtuple('Organization', 'name rating reviews mean_check categories address')

In [52]:
for i in atm.index:
    for key, value in to_replace.items():
        atm.at[i,'address'] =  atm.at[i,'address'].replace(key, value)

In [55]:
to_replace = {
     'AV.': ' PROSPECT ',
     'AVE': ' PROSPECT ', 
     'PR-KT': ' PROSPECT ', 
     'STR.': ' ULITSA ',
     ' ST ': ' ULITSA ',
     'UL.' : ' ULITSA ',
     'H/W.': ' SHOSSE ',
     ', LIT. ': '',
     'D.': '',
     'VLAD. ':'',
     ', KORP. ': '/',
     ' DZERZHINSK':  ' DZERZHINSKOGO ',
     '.': '. ',
     ',': ', ',
}

In [140]:
def parsing(atms, start):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i in  tqdm_notebook(atms.index[start:]):
        urli = url + ' '.join([atms.at[i,'address'], atms.at[i,'city']]).replace(' ', '%20')
        driver.get(urli)
        # Широта и долгота 
        try:
            element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
            atms.at[i,'lat'],  atms.at[i,'long'] = list(map(float, element.text.split(',')))
        except: pass
        
        # Адрес 
        try:
            addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']").text
        except:  addr_part1 = False
            
        try:
            addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']").text
        except:  addr_part2 = False
        
        atms.at[i, 'address_rus'] = '%s, %s' % (addr_part1, addr_part2)


In [139]:
pickle.dump(atm[:4282],open('atm4282.dt','wb'))

In [142]:
pickle.dump(atm[:4283+1382],open('atm5.dt','wb'))

In [148]:
pickle.dump(atm,open('atm_.dt','wb'))

In [145]:
4283+1382+ 910

6575

In [155]:
7596 - sum(atm['small_len'])

6143

In [149]:
atm['small_len'] = atm.address_rus.map(lambda x: len(x)) < 25

In [167]:
sum(atm.long.isnull())

1980

In [169]:
7596 - 1980

5616

In [168]:
len(atm)

7596

In [161]:
atm.loc[atm['small_len'] == True,'address_rus'] = None
atm.loc[atm['small_len'] == True,'lat'] = None
atm.loc[atm['small_len'] == True,'long'] = None

In [163]:
atm.to_excel('Банкоматы.xlsx')

In [171]:
atm[atm.long.isnull()]

,atm_caid,atm_terminal,full_address,address,city,address_rus,lat,long,small_len
2,VB24,388084,"D. 26/24, PR-KT INDUSTRSANKT-PETERBU","26/24, PROSPECT INDUSTR",SAINT PETERSBURG,None,None,None,True
7,GAZPROMBANK,405289,GAL. AEROPORT MOSKVA,GAL. AEROPORT,MOSCOW,None,None,None,True
13,Alfa Iss,156058,4 LOKOMOTIVNIY PASS. MOSCOW,4 LOKOMOTIVNIY PASS.,MOSCOW,None,None,None,True
17,Alfa Iss,502870,9B RABOCHE-KRESTYAN VOLGOGRAD,9B RABOCHE-KRESTYAN,VOLGOGRAD,None,None,None,True
19,GAZPROMBANK,406943,"27 VOLG,EREMENKO.54 VOLGOGRAD","27 VOLG, EREMENKO. 54",VOLGOGRAD,None,None,None,True
29,Alfa Iss,220202,2 AKSAYSKIY AVE AKSAY,2 AKSAYSKIY PROSPECT,AKSAY,None,None,None,True
30,Alfa Iss,220084,5/60 339-STRELKOVAYA DIROSTOV-ON-DON,5/60 339-STRELKOVAYA DI,ROSTOV-ON-DON,None,None,None,True
47,VB24,387424,"D. 59, KORP. 1, UL. PARMOSKVA G","59/1, ULITSA PAR",MOSCOW,None,None,None,True
55,Alfa Iss,155996,24 AK.LAZAREVA STR. MOSCOW,24 AK. LAZAREVA ULITSA,MOSCOW,None,None,None,True
58,Alfa Iss,220256,239 SOVETSKOY ARMII SAMARA,239 SOVETSKOY ARMII,SAMARA,None,None,None,True


In [146]:
parsing(atm, 6575)

In [ ]:
  #Допарсить остатки
    try:
            if not addr_part1  or not addr_part2:
                print(driver.find_element_by_xpath("//div[@class='card-feature-view _subcontent']").text)
                atms.at[i, 'address_rus'] = driver.find_element_by_xpath("//div[@class='card-feature-view _subcontent']").text
        except: pass

In [88]:
    SEARCHING_MAP = {'Еда': 'eat',
                      'Авто': 'auto',
                      'Покупки': 'purchase',
                      'Красота': 'beauty',
                      'Услуги': 'service',
                      'Развлечения и отдых': 'entertainment',
                      'Спорт': 'sport',
                      'Религия': 'religion', 
                      'Транспорт': 'transport',
                      'Госуслуги': 'government',
                      'Деньги': 'money',
                      'Образование': 'education',
                      'Стройка и ремонт': 'repair',
                      'Путешествия': 'travel'}

In [32]:
for atm in atms: 
    setattr(atm, 'address_', atm.full_address[:23].strip())
    setattr(atm, 'city_', atm.full_address[23:].strip())

In [97]:
for atm in atms: 
    for key, value in to_replace.items():
        atm.address_ = atm.address_.replace(key, value)
        # setattr(atm, 'address_', atm.address_.replace(key, value))

In [89]:
class ImportantObject():
    def __init__(self):
        self.name = None
        self.lat, self.long = None, None # Широта и долгота
        self.types = list()  # Типы объекта по внутренней классификации Яндекс.Карт
        self.score = None # "Народная" оценка уровня ослуживания 
        self.link = None # Ссылка на объект 
        self.address = None  # Адрес  

In [73]:
Station = namedtuple('Station', 'name distance')
Organization = namedtuple('Organization', 'name rating reviews mean_check categories address')

class ATM():
    """
    Объект "Банкомат"
    """
    SEARCHING_MAP = {'Еда': 'eat',
                      'Авто': 'auto',
                      'Покупки': 'purchase',
                      'Красота': 'beauty',
                      'Услуги': 'service',
                      'Развлечения и отдых': 'entertainment',
                      'Спорт': 'sport',
                      'Религия': 'religion', 
                      'Транспорт': 'transport',
                      'Госуслуги': 'government',
                      'Деньги': 'money',
                      'Образование': 'education',
                      'Стройка и ремонт': 'repair',
                      'Путешествия': 'travel'}
    
    def __init__(self, atm_caid, atm_terminal, full_address):
        self.atm_caid = atm_caid # Идентификатор банковской структуры, которой принадлежит банкомат
        self.atm_terminal = atm_terminal # Идентификатор банкомата
        self.full_address = full_address # Адрес в формате хранения в информационной системе банка
        self.address_ = self.full_address[:23].strip()
        self.city_ = self.full_address[23:].strip()
        
        self.lat, self.long = None, None # Широта и долгота
        self.full_address_rus = None # Адрес полученнный в процессе парсинга с использованием ресурса Яндекс.Карты
        self.metro_stations = list() # Ближайшие метро и расстояния до них
        self.bus_stations = list() # Ближайшие остановки общественного транспорта и расстояния до них 
        self.inbuilding = list() # Объекты располагающиеся в том же здании, что и банкомат.
        
        self.eat = list() # Еда. Ближайшие к банкомату кафе и рестораны. Список объектов типа ImportantObject
        self.auto = list()
        self.purchase = list()
        self.service = list()
        self.entertainment = list()
        self.sport = list()
        self.religion = list()
        self.transport = list()
        self.government = list()
        self.money = list()
        self.education = list()
        self.repair = list()
        self.travel = list()
         
    @property
    def address(self):
        return self.address_
    
    @address.setter
    def address(self, value):
        self.address_ = value
    
    @property
    def city(self):
        return self.city_
    
    @city.setter
    def city(self, value):
        self.city_ = value
    
    def distance(self, org: Organization):
        pass    

In [13]:
class ATM():
    """
    Объект "Банкомат"
    """
    SEARCHING_MAP = {'Еда': 'eat',
                      'Авто': 'auto',
                      'Покупки': 'purchase',
                      'Красота': 'beauty',
                      'Услуги': 'service',
                      'Развлечения и отдых': 'entertainment',
                      'Спорт': 'sport',
                      'Религия': 'religion', 
                      'Транспорт': 'transport',
                      'Госуслуги': 'government',
                      'Деньги': 'money',
                      'Образование': 'education',
                      'Стройка и ремонт': 'repair',
                      'Путешествия': 'travel'}
    
    def __init__(self, atm_caid, atm_terminal, full_address):
        self.atm_caid = atm_caid # Идентификатор банковской структуры, которой принадлежит банкомат
        self.atm_terminal = atm_terminal # Идентификатор банкомата
        self.full_address = full_address # Адрес в формате хранения в информационной системе банка 
        
        self.lat, self.long = None, None # Широта и долгота
        self.full_address_rus = None # Адрес полученнный в процессе парсинга с использованием ресурса Яндекс.Карты
        self.metro_stations = list() # Ближайшие метро и расстояния до них
        self.bus_stations = list() # Ближайшие остановки общественного транспорта и расстояния до них 
        self.inbuilding = list() # Объекты располагающиеся в том же здании, что и банкомат.
        
        self.eat = list() # Еда. Ближайшие к банкомату кафе и рестораны. Список объектов типа ImportantObject
        self.auto = list()
        self.purchase = list()
        self.service = list()
        self.entertainment = list()
        self.sport = list()
        self.religion = list()
        self.transport = list()
        self.government = list()
        self.money = list()
        self.education = list()
        self.repair = list()
        self.travel = list()
         
    @property
    def address(self):
        return self.full_address[:23].strip()
    
    @property
    def city(self):
        return self.full_address[23:].strip()
    
    def distance(self, org: Organization):
        pass    

In [90]:
atms = [ATM(atmi.atm_caid, atmi.atm_terminal, atmi.full_address) for i, atmi in atm.iterrows()] 

In [92]:
def parsing(atms):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atm in  tqdm_notebook(enumerate(atms)):
        urli = url + ' '.join([atm.address, atm.city]).replace(' ', '%20')
        driver.get(urli)
        try: 
            # Широта и долгота 
            element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
            atm.lat,  atm.long = list(map(float, element.text.split(',')))
            
            # Адрес 
            try:
                addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']")
            except:  addr_part1 = ''
            try:
                addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']")
            except:  addr_part2 = ''
            #print(addr_part1.text, addr_part2.text) 
            atm.full_address_rus = '%s, %s' % (addr_part1.text, addr_part2.text) 
            
            # Прасинг ближайших станций метро и автобусных остановок
            # parse_station(driver, atm)

            # Парсинг организаций находящихся в здании 
            # try:
            #    parse_inbuilding(driver, atm)
            # except: pass

            # Парсинг организаций оказывающих услуги в сфере питания
            #parse_organization(driver, atm)
            pickle.dump(atm, open('D:/atm_data_2/atm_%s_%s.dat'%(atm.atm_caid, atm.atm_terminal), 'wb'))
        except:
            pass

In [ ]:
parsing(atms)

In [19]:
bad_parsing = [atm for atm in atms if len(atm.full_address_rus or '') < 25]

In [23]:
to_replace = {'AV.': ' PROSPECT ',
             'AVE': ' PROSPECT ', 
             'PR-KT': ' PROSPECT ', 
             'STR.': ' ULITSA ',
             ' ST ': ' ULITSA ',
             'UL.' : ' ULITSA ',
             'H/W.': ' SHOSSE ',
             ', LIT. ': '',
             'D.': '',
             'VLAD. ':'',
             ', KORP. ': '/',
             ' DZERZHINSK':  ' DZERZHINSKOGO ',
            }

In [28]:
for atm in bad_parsing: 
    for key, value in to_replace.items():
        atm.address = atm.address.replace(key, value)

AttributeError: can't set attribute

In [ ]:
url  = 'https://yandex.ru/maps/213/moscow/?ll=37.628056%2C55.744059&z=16&mode=search&ol=geo&ouri=ymapsbm1%3A%2F%2Fgeo%3Fll%3D37.628063%252C55.744061%26spn%3D0.001000%252C0.001000%26text%3D11%252F23%252C%2520Pyatnitskaya%2520Street%252C%2520Moscow%252C%2520Russia%2520'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

In [91]:
def parse_inbuilding(driver, atm):
    # Переход на страницу содержащую список объектов находящихся в здании с банкоматом
    sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__hint _interactive']")
    sections[0].click()
    
    elements_org = get_elements(driver)

    # Записываем результаты в объект
    atm.inbuilding = [parse_one_organization(element) for element in elements_org] 
    
    # возвращение на исходную станицу 
    driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")[0].click()

    
def parse_station(driver, atm):
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__count']")
    lens = [int(x.text)+1 for x in sections]
    
    # раскрываем выпадающие списки
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__control']")
    for x in sections[::-1]: x.click()

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-name']")
    names = [x.text for x in sections] 

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-distance']")
    distances = [x.text for x in sections] 
    
    if len(lens) == 1: 
        atm.bus_stations = [Station(name, distance) for name, distance in  zip(names, distances)]
    elif len(lens) == 2:
        atm.metro_stations  =  [Station(name, distance) for name, distance in list(zip(names, distances))[:lens[0]]]
        atm.bus_stations =  [Station(name, distance) for name, distance in list(zip(names, distances))[lens[0]:]]
    else:
        pass   
    
def get_elements(driver):
        # Парсинг элементов списка содержащих информацию о близлежаийших организациях  
        elements = list()
        while True: 
            time.sleep(1)
            len_ = len(elements)
            elements = driver.find_elements_by_xpath("//ul[@class='search-page-view']")
            scroll_element = elements[-1].find_elements_by_xpath(".//li") 
            scroll_element[-1].location_once_scrolled_into_view
            if len_ == len(elements):
                break
                
        result = []
        for x in [element.find_elements_by_xpath("./*") for element in elements]: result.extend(x)
        return result

def clear_input(element):
    element.send_keys(Keys.CONTROL + "a");
    element.send_keys(Keys.DELETE);

def parse_organization(driver, atm):
    for search_string, attr in SEARCHING_MAP.items():
        element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
        time.sleep(2)
        clear_input(element)
        ActionChains(driver).click(element).send_keys(search_string, Keys.ENTER).perform()
        time.sleep(0.5)
        # Парсинг элементов содержащих информацию о близлежаийших организациях  
        elements_org = get_elements(driver)

        # Записываем результаты в объект
        result = [parse_one_organization(element) for element in elements_org] 
        setattr(atm, attr, result)

        element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
        clear_input(element)
        time.sleep(0.5)
        ActionChains(driver).click(element).send_keys(atm.full_address, Keys.ENTER).perform()

    

def parse_one_organization(element):
        name, rating, reviews, mean_check, categories, address = [None]*6
        try:
            name = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__header']/a[1]")[0].text
        except: pass  
        
        try:
            #rating = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__rating']")[0].text
            rating = element.find_elements_by_xpath(".//span[@class='business-rating-badge-view__rating']")[0].text
        except: pass

        try:
            reviews = element.find_elements_by_xpath(".//span[@class='business-reviews-count-view']")[0].text
        except: pass

        try:
            mean_check = element.find_elements_by_xpath(".//div[@class='search-business-snippet-subtitle-view']")[0].text
        except: pass

        try:
            categories_elms = element.find_elements_by_xpath(".//div[@class='search-snippet-categories-view']/*")
            categories  = [category.text for category in categories_elms]    
        except: pass

        try:
            address = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__address']")[0].text
        except: pass 
        return Organization(name, rating, reviews, mean_check, categories, address)

In [ ]:
def parse_inbuilding(driver, atm):
    # sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__show-all-link']")
    sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__hint _interactive']")
    sections[0].click()
    
    elements = driver.find_elements_by_xpath("//ul[@class='search-page-view']//li")
    
    sections = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title']")
    names = [x.text for x in sections] 

    sections = driver.find_elements_by_xpath("//div[@class='search-business-snippet-view__categories']")
    categoties = [x.text for x in sections] 
    
    atm.inbuilding = [Organization(name, category) for name, category in zip(names, categoties)]
    
    # возвращение на исходную станицу
    sections = driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")
    sections[0].click()

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
driver.get(url)

In [ ]:
atm = ATM(atm_caid=1, atm_terminal=2, full_address='sdfsafsdfsafsafsafsafdasdfasfsafs')

In [ ]:
atm.address_rus = 'Пятницкая улица, 11/23, Москва, Россия, 115184'

In [ ]:
for search_string, attr in list(SEARCHING_MAP.items())[:4]:
    print(attr)
    element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
    # print(element.get_attribute("value"))
    time.sleep(2)
    element.send_keys(Keys.CONTROL + "a");
    element.send_keys(Keys.DELETE);
    time.sleep(2)
    #ActionChains(driver).click(element).send_keys(search_string, Keys.ENTER).perform()
    ActionChains(driver).click(element).send_keys(atm.address_rus, Keys.ENTER).perform()
    time.sleep(2)

In [ ]:
element = driver.find_elements_by_xpath("//input[@class='input_air-search-large__control']")[0]
ActionChains(driver).click(element).send_keys('Еда', Keys.ENTER).perform()

In [ ]:
elements = list()
while True: 
    time.sleep(0.5)
    len_ = len(elements)
    elements = driver.find_elements_by_xpath("//ul[@class='search-page-view']")
    scroll_element = elements[-1].find_elements_by_xpath(".//li") 
    scroll_element[-1].location_once_scrolled_into_view
    if len_ == len(elements):
        break

In [ ]:
elements_org = []
for x in [element.find_elements_by_xpath("./*") for element in elements]: elements_org.extend(x)

In [ ]:
result = list()
for element in elements_org: 
    name, rating, reviews, mean_check, categories, address = [None]*6
    name = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__header']/a[1]")[0].text
    try:
        #rating = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__rating']")[0].text
        rating = element.find_elements_by_xpath(".//span[@class='business-rating-badge-view__rating']")[0].text
    except: pass
    
    try:
        reviews = element.find_elements_by_xpath(".//span[@class='business-reviews-count-view']")[0].text
    except: pass
    
    try:
        mean_check = element.find_elements_by_xpath(".//div[@class='search-business-snippet-subtitle-view']")[0].text
    except: pass
    
    try:
        categories_elms = element.find_elements_by_xpath(".//div[@class='search-snippet-categories-view']/*")
        categories  = [category.text for category in categories_elms]    
    except: pass
    
    try:
        address = element.find_elements_by_xpath(".//div[@class='search-business-snippet-view__address']")[0].text
    except: pass 
    org = Organization(name, rating, reviews, mean_check, categories, address)
    result.append(org)

In [ ]:
def parse_inbuilding(driver, atm):
    sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__show-all-link']")
    sections[0].click()
    
    sections = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title']")
    names = [x.text for x in sections] 

    sections = driver.find_elements_by_xpath("//div[@class='search-business-snippet-view__categories']")
    categoties = [x.text for x in sections] 
    
    atm.inbuilding = [Organization(name, category) for name, category in zip(names, categoties)]
    
    # возвращение на исходную станицу
    sections = driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")
    sections[0].click()

In [ ]:
def parse_station(driver, atm):
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__count']")
    lens = [int(x.text)+1 for x in sections]
    
    # раскрываем выпадающие списки
    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__control']")
    for x in sections[::-1]: x.click()

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-name']")
    names = [x.text for x in sections] 

    sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-distance']")
    distances = [x.text for x in sections] 
    
    if len(lens) == 1: 
        atm.bus_stations = [Station(name, distance) for name, distance in  zip(names, distances)]
    elif len(lens) == 2:
        atm.metro_stations  =  [Station(name, distance) for name, distance in zip(names, distances)[:lens[0]]]
        atm.bus_stations =  [Station(name, distance) for name, distance in zip(names, distances)[lens[0]:]]
    else:
        pass

In [ ]:
atm

In [ ]:
parsing(atm)

In [ ]:
atm1 = atm[atm.lat.isnull() & atm.atm_caid.isin({'VB24','GAZPROMBANK', 'Alfa Iss', 'Alfa Acq'})]

In [ ]:
url  = 'https://yandex.ru/maps/213/moscow/?ll=37.628056%2C55.744059&z=16&mode=search&ol=geo&ouri=ymapsbm1%3A%2F%2Fgeo%3Fll%3D37.628063%252C55.744061%26spn%3D0.001000%252C0.001000%26text%3D11%252F23%252C%2520Pyatnitskaya%2520Street%252C%2520Moscow%252C%2520Russia%2520'

In [ ]:
url2 = 'https://yandex.ru/maps/10668/murom/?ll=41.973135%2C55.526857&z=16&mode=search&text=D.%2018%2C%20UL.%20LENINGRADSKA%20MUROM&sll=37.657682%2C55.740882&sspn=0.023303%2C0.006283'

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')

In [ ]:
url  = 'https://yandex.ru/maps/213/moscow/?ll=37.628056%2C55.744059&z=16&mode=search&ol=geo&ouri=ymapsbm1%3A%2F%2Fgeo%3Fll%3D37.628063%252C55.744061%26spn%3D0.001000%252C0.001000%26text%3D11%252F23%252C%2520Pyatnitskaya%2520Street%252C%2520Moscow%252C%2520Russia%2520'
driver.get(url)

## Парсинг остановок и метро

In [ ]:
sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__count']")
lens = [int(x.text)+1 for x in sections]

sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='card-dropdown-view__control']")
# раскрываем выпадающие списки
for x in sections[::-1]: x.click()

sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-name']")
names = [x.text for x in sections] 

sections = driver.find_elements_by_xpath("//div[@class='masstransit-stops-view']//div[@class='masstransit-stops-view__stop-distance']")
distances = [x.text for x in sections] 

if len(lens) == 1: 
    stations = list(zip(names, distances))
else:
    metro  = list(zip(names, distances))[:lens[0]]
    bus_stations = list(zip(names, distances))[lens[0]:]

## Парсинг бизнес объектов  если их меньше пяти

In [ ]:
sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view']//div[@class='card-snippet-view__title']")

In [ ]:
businesses_names = [x.text for x in sections] 

In [ ]:
businesses_names

In [ ]:
sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view']//div[@class='card-snippet-view__description']")

In [ ]:
businesses_categoties = [x.text for x in sections] 

In [ ]:
businesses  = list(zip(businesses_names, businesses_categoties))

## Парсинг бизнес объектов  если их больше пяти

In [ ]:
sections = driver.find_elements_by_xpath("//div[@class='businesses-list-view__show-all-link']")

sections[0].click()

sections = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title']")

businesses_names = [x.text for x in sections] 

sections = driver.find_elements_by_xpath("//div[@class='search-business-snippet-view__categories']")

businesses_categoties = [x.text for x in sections] 

businesses = list(zip(businesses_names, businesses_categoties))

## Переход обратно со страницы бизнес объектов

In [ ]:
sections = driver.find_elements_by_xpath("//div[@class='sidebar-panel-header-back-view__icon']")

sections[0].click()

In [ ]:
driver.find_elements_by_xpath("//div[@class='catalog-small-item-view__icon']")[0].click()

sections = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title']")

eat_names = [x.text for x in sections] 

eat_names

sections = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title _with-badge']")

eat_names = [x.text for x in sections] 

eat_names

In [ ]:
card-snippet-view__description

In [ ]:
get_html('https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=')

In [ ]:
for i, atmi in log_progress(atm1.iterrows()):
    urli = url + atmi.adress_eng.replace(' ', '%20')
    driver.get(urli)
    try: 
        element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
        atm.at[i, 'lat'],  atm.at[i, 'long'] = list(map(float, element.text.split(',')))
        
        addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']")
        addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']")
        atm.at[i, 'adress'] = addr_part1.text + ', ' + addr_part2.text 
    except:
        pass

In [ ]:
def get_html(url):
    response = urllib.request.urlopen(url)
    return response.read()

In [ ]:
pickle.dump(atm1,open('atm1.df','wb'))

In [ ]:
for i, atmi in log_progress(atm.iterrows()):
    urli = url + atmi.adress_eng.replace(' ', '%20')
    html = get_html(urli)
    soup = BeautifulSoup(html)
    try: 
        coord = soup.find('div', class_='clipboard__action-wrapper _inline').text
        atm.at[i, 'lat'],  atm.at[i, 'long'] = list(map(float, coord.split(',')))

        coord = soup.find('div', class_='card-title-view__title').text
        coord = soup.find('div', class_='toponym-card-view__description').text

        atm.at[i, 'adress'] = addr_part1.text + ', ' + addr_part2.text 
    except:
        pass